<a href="https://colab.research.google.com/github/eynomr/Course-Planner/blob/main/CPSC481_Project_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations

Problem Class Definition

In [4]:
class Problem(object):
    """
    The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states
    (or give an `is_goal` method) and perhaps other keyword args for the subclass.
    """

    def __init__(self, initial=None, goal=None, **kwds):
        self.__dict__.update(initial=initial, goal=goal, **kwds)

    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

Node Class Definition

In [5]:
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost


failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.

In [6]:
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)


def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None):
        return []
    return path_states(node.parent) + [node.state]

In [7]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x):
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)

    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]

    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

Search Functions

In [8]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def best_first_tree_search(problem, f):
    "A version of best_first_search without the `reached` table."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n): return n.path_cost


def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n), with no `reached` table."""
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

In [35]:
df = pd.read_csv('./data/course_data.csv')

In [36]:
df

,course_number,course_name,description,work_load,prerequisits,required,type
0,CPSC 120,Introduction to Programming (3),Introduction to the concepts underlying all co...,"1.5 hours lecture, 3 hours laboratory",MATH 125,1,lower_division
1,CPSC 121,Object-Oriented Programming (3),The object-oriented programming paradigm: clas...,"2 hours lecture, 2 hours activity",CPSC 120,1,lower_division
2,CPSC 131,Data Structures (3),"Classical data structures: vector, linked list...","Classical data structures: vector, linked list...",CPSC 121,1,lower_division
3,CPSC 223C,C Programming (3),"Systems programming in the C language, includi...","Systems programming in the C language, includi...",CPSC 131,1,lower_division
4,CPSC 223J,Java Programming (3),"Characteristics of Java: portable, robust, sec...","2 hours lecture, 2 hours lab per week",CPSC 131,1,lower_division
...,...,...,...,...,...,...,...
75,MATH 335,Mathematical Probability (4),"Probability theory; discrete, continuous and m...","Probability theory; discrete, continuous and m...",MATH 250A,0,other
76,MATH 340,Numerical Analysis (4),Approximate numerical solutions of systems of ...,Approximate numerical solutions of systems of ...,"MATH 250A, MATH 207 or MATH 250B, MATH 107, MA...",0,other
77,MATH 370,Mathematical Model Building (4),Introduction to mathematical models in science...,Introduction to mathematical models in science...,"MATH 250A, MATH 207 or MATH 250B, MATH 107, MA...",0,other
78,HONR 201B,Honors Seminar: American Institutions and Valu...,Critically examines the development of America...,Critically examines the development of America...,NaN,0,other


In [40]:
df['course_units'] = df['course_name'].str.extract(r'\((\d+)\)')
course_data = df[['course_number', 'course_units', 'prerequisits', 'required', 'type']]
course_data = course_data[course_data['required'] == True]

courses = dict(zip(course_data['course_number'], course_data['course_units']))
prereqs = {}
for index, row in course_data.iterrows():
  course_number = row['course_number']
  prereq_str = row['prerequisits']
  prereqs[course_number] = [] if pd.isna(prereq_str) else prereq_str.split(',')

print(prereqs)

{'CPSC 120': ['MATH 125'], 'CPSC 121': ['CPSC 120'], 'CPSC 131': ['CPSC 121'], 'CPSC 223C': ['CPSC 131'], 'CPSC 223J': ['CPSC 131'], 'CPSC 223N': ['CPSC 131'], 'CPSC 223P': ['CPSC 131'], 'CPSC 223W': ['CPSC 131'], 'CPSC 240': ['CPSC 131', ' MATH 170A or MATH 280'], 'CPSC 253': [], 'CPSC 315': ['CPSC 131'], 'CPSC 323': ['CPSC 131'], 'CPSC 332': ['CPSC 131'], 'CPSC 335': ['CPSC 131', ' MATH 170A', ' MATH 150A'], 'CPSC 351': ['CPSC 131'], 'CPSC 362': ['CPSC 131'], 'CPSC 471': ['CPSC 351'], 'CPSC 481': ['CPSC 335', ' MATH 338'], 'CPSC 490': ['CPSC 362'], 'CPSC 491': ['CPSC 490'], 'MATH 150A': [], 'MATH 150B': ['MATH 150A'], 'MATH 170A': [], 'MATH 170B': [], 'MATH 338': ['MATH 106', ' MATH 130 or MATH 150B'], 'CPSC 254': ['CPSC 131'], 'CPSC 301': ['CPSC 131'], 'CPSC 349': ['CPSC 131'], 'CPSC 352': ['MATH 170B', ' CPSC 131', ' CPSC 253'], 'CPSC 375': ['CPSC 131', ' MATH 338'], 'CPSC 386': ['CPSC 121'], 'CPSC 411': ['CPSC 131'], 'CPSC 411A': ['CPSC 131'], 'CPSC 431': ['CPSC 332'], 'CPSC 439':

In [50]:
# need to check for each type if the total units are met (lower, upper, math, electives)

class CoursePlanner(Problem):
  def __init__(self, courses, initial=None, goal=None):
    Problem.__init__(self, initial=initial, goal=goal, courses=courses)

  def actions(self, state):
    available_courses = [course for course in self.courses if state[course] == 0]
    valid_actions = []

    for course in available_courses:
      # Check if prereq is satisfied
      prereq_satisfied = all(prereq in state and state[prereq] > 0 for prereq in prereqs[course])

      if prereq_satisfied:
        # check if max units are reached
        if state['units'] + int(courses[course]) <= 17:
          valid_actions.append(course)

    return valid_actions


    def results(self, state, action):
      # take new courses and return the new state
      new_state = state.copy()

      # set the course as taken
      new_state[action] = 1
      new_state['units'] += int(courses[action])
      return new_state

    def is_goal(self, state):
      return all(state[course] == 1 for course in self.courses)

    def h(self, node):
      # gotta think about this one
      return 0



In [44]:
initial_state = {course: 0 for course in courses}
initial_state['units'] = 0
print(initial_state)
goal_state = {course: 1 for course in courses}

{'CPSC 120': 0, 'CPSC 121': 0, 'CPSC 131': 0, 'CPSC 223C': 0, 'CPSC 223J': 0, 'CPSC 223N': 0, 'CPSC 223P': 0, 'CPSC 223W': 0, 'CPSC 240': 0, 'CPSC 253': 0, 'CPSC 315': 0, 'CPSC 323': 0, 'CPSC 332': 0, 'CPSC 335': 0, 'CPSC 351': 0, 'CPSC 362': 0, 'CPSC 471': 0, 'CPSC 481': 0, 'CPSC 490': 0, 'CPSC 491': 0, 'MATH 150A': 0, 'MATH 150B': 0, 'MATH 170A': 0, 'MATH 170B': 0, 'MATH 338': 0, 'CPSC 254': 0, 'CPSC 301': 0, 'CPSC 349': 0, 'CPSC 352': 0, 'CPSC 375': 0, 'CPSC 386': 0, 'CPSC 411': 0, 'CPSC 411A': 0, 'CPSC 431': 0, 'CPSC 439': 0, 'CPSC 440': 0, 'CPSC 449': 0, 'CPSC 454': 0, 'CPSC 455': 0, 'CPSC 456': 0, 'CPSC 458': 0, 'CPSC 459': 0, 'CPSC 462': 0, 'CPSC 463': 0, 'CPSC 464': 0, 'CPSC 466': 0, 'CPSC 474': 0, 'CPSC 479': 0, 'CPSC 483': 0, 'CPSC 484': 0, 'CPSC 485': 0, 'CPSC 486': 0, 'CPSC 489': 0, 'CPSC 499': 0, 'EGGN 495': 0, 'units': 0}


In [51]:
course_planner = CoursePlanner(courses, initial_state, goal_state)

In [52]:
solution = astar_search(course_planner)

TypeError: unhashable type: 'dict'